<a href="https://colab.research.google.com/github/ahdali24/SP-XD/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# -*- coding: utf-8 -*-
# جاهز للتشغيل في Colab / Kaggle / Local
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import joblib

# === مسار الملف - عدّليه لو لازم ===
CSV_PATH = "/content/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"

# === قراءة الملف ===
df = pd.read_csv(CSV_PATH, sep=',')
df.columns = df.columns.str.strip()   # تنظيف المسافات حول أسماء الأعمدة
print("عدد الصفوف:", len(df))

# === 1) نشوف كل الفئات المتاحة في العمود Label (أو أي اسم مطابق) ===
df.columns = df.columns.str.strip()
label_col = "Label"
print(df[label_col].value_counts())


print("\nعمود الlabel المستخدم:", label_col)
print("\nقيمة_counts للـ labels (أول 50):")
print(df[label_col].astype(str).str.strip().value_counts().head(50))

# === 2) اكتشاف تسميات PortScan تلقائياً ===
# نبحث عن أي label تحتوي على كلمة 'port' أو 'scan' (case-insensitive)
labels = df[label_col].astype(str).str.strip()
portscan_mask = labels.str.lower().str.contains("port") | labels.str.lower().str.contains("scan")
portscan_labels = sorted(labels[portscan_mask].unique().tolist())
print("\nالتسميات التي تم اعتبارها PortScan (مكتشفة تلقائياً):", portscan_labels)

if len(portscan_labels) == 0:
    # لو مفيش، خلي المستخدم يحدد اسم الهجوم يدوياً
    raise ValueError("لم أجد أي تسميات تبدو كـ PortScan. شغّل print(df[label_col].value_counts()) وتأكد من الأسماء.")

# === 3) نعمل target binary: 1 = PortScan، 0 = باقي ===
y = labels.apply(lambda x: 1 if x in portscan_labels else 0)
print("\nتوزيع الفئات (PortScan vs Others):", dict(zip(*np.unique(y, return_counts=True))))

# === 4) اختيار الميزات (تأكدي تعديل الأسماء لو عندك اختلاف) ===
base_features = [
    "Destination Port", "Flow Duration",
    "Init_Win_bytes_forward", "Init_Win_bytes_backward",
    "Subflow Fwd Bytes", "Subflow Bwd Bytes",
    "act_data_pkt_fwd", "min_seg_size_forward",
    # شوية أعمدة شائعة في CIC-IDS2017 (لو موجودة)
    "Total Fwd Packets", "Total Bwd Packets", "Flow Bytes/s", "Flow Packets/s",
    "Total Length of Fwd Packets", "Total Length of Bwd Packets"
]
feature_cols = [c for c in base_features if c in df.columns]
print("\nالميزات المتاحة من القالب:", feature_cols)

# === 5) Feature engineering إضافية ===
X = df[feature_cols].copy()

# 5.1 مجموع الباكتس لكل فلو لو موجودين
if "Total Fwd Packets" in df.columns and "Total Bwd Packets" in df.columns:
    X["packets_per_flow"] = pd.to_numeric(df["Total Fwd Packets"], errors='coerce').fillna(0) + \
                            pd.to_numeric(df["Total Bwd Packets"], errors='coerce').fillna(0)

# 5.2 bytes per packet إن أمكن
# نحاول استخدام Flow Bytes/s أو مجموع الأطوال
if "Flow Bytes/s" in df.columns and "packets_per_flow" in X.columns:
    # لاحظ: Flow Bytes/s قد يكون معدل، لكن نجرب حساب bytes_per_packet بتقدير بسيط
    X["bytes_per_packet"] = pd.to_numeric(df.get("Flow Bytes/s", 0), errors='coerce') / (X["packets_per_flow"].replace(0, np.nan))
else:
    # بالتناوب: استخدام مجموع أطوال الـ fwd/bwd مقسوم على packets
    if "Total Length of Fwd Packets" in df.columns and "Total Length of Bwd Packets" in df.columns and "packets_per_flow" in X.columns:
        X["bytes_per_packet"] = (pd.to_numeric(df["Total Length of Fwd Packets"], errors='coerce').fillna(0) +
                                 pd.to_numeric(df["Total Length of Bwd Packets"], errors='coerce').fillna(0)) / X["packets_per_flow"].replace(0, np.nan)

# 5.3 unique_dst_ports_per_src (لو فيه Source IP و Destination Port)
src_ip_cols = [c for c in df.columns if c.lower().replace(" ", "") in ("sourceip","srcip","source ip","src ip","source_ip","src_ip")]
dst_port_col = None
for name in ["Destination Port", "Dst Port", "DstPort", "DestinationPort", "Dst Port Number"]:
    if name in df.columns:
        dst_port_col = name
        break
if len(src_ip_cols) > 0 and dst_port_col is not None:
    src_col = src_ip_cols[0]
    print("\nاستخدام عمود الـ Source IP:", src_col)
    # نعمل aggregation: لكل source ip نعد عدد المنافذ المقصودة المختلفة خلال الdataset
    tmp = df[[src_col, dst_port_col]].copy()
    # نعالج القيم المفقودة
    tmp = tmp.dropna(subset=[src_col, dst_port_col])
    ports_per_src = tmp.groupby(src_col)[dst_port_col].nunique().rename("unique_dst_ports_per_src")
    # نربطه بالـ X عبر merge على الفهرس الأصلي
    ports_per_src_df = tmp.merge(ports_per_src.reset_index(), on=src_col, how='left')
    # نحاول الانضمام: أفضل طريقة هي map من الـ src_col للقيمة
    map_ports = ports_per_src.to_dict()
    X["unique_dst_ports_per_src"] = df[src_col].map(map_ports).fillna(0)
else:
    print("لم يتم العثور على Source IP أو Destination Port بأسماء متوقعة -> سيتم تجاهل unique_dst_ports_per_src")

# === 6) تحويل لأرقام والتعامل مع القيم الفارغة ===
X = X.apply(pd.to_numeric, errors='coerce')
print("\nنسبة القيم الفارغة لكل feature قبل التعويض:")
print(X.isna().mean())

# نعوّض بالقيمة المتوسطة لكل عمود
X = X.fillna(X.mean())

print("\nبعد التعويض، أي قيم فارغة؟", X.isna().sum().sum())

# === 7) Scaling ===
# معالجة القيم اللانهائية والقيم الكبيرة جدًا
X = X.replace([np.inf, -np.inf], np.nan)
X = X.fillna(X.mean())

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# === 8) التعامل مع عدم التوازن ===
# لو PortScan قليل جدا نستخدم SMOTE، وإلا نستخدم undersample للفئة الكبيرة
from collections import Counter
print("\nتوزيع الفئات قبل resampling:", Counter(y))
minor_count = Counter(y)[1]
major_count = Counter(y)[0]

if minor_count == 0:
    raise ValueError("لا توجد أمثلة لفئة PortScan بعد الفحص التلقائي.")
if minor_count < 0.1 * major_count:
    print("الفئة الهجومية قليلة - سنستخدم SMOTE لعمل oversample")
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X_scaled, y)
else:
    print("الفئات متقاربة نسبياً أو الفئة الهجومية ليست قليلة جداً - سنستخدم undersample بسيط للفئة الكبرى")
    rus = RandomUnderSampler(random_state=42)
    X_res, y_res = rus.fit_resample(X_scaled, y)

print("شكل البيانات بعد الresample:", X_res.shape)
print("توزيع الفئات بعد الresample:", dict(zip(*np.unique(y_res, return_counts=True))))

# === 9) split + train ===
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.25, random_state=42, stratify=y_res)

model = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# === 10) تقييم ===
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))

# نعرض أهم الميزات
try:
    importances = model.feature_importances_
    feat_names = X.columns.tolist()
    feat_imp_df = pd.DataFrame({"feature": feat_names, "importance": importances}).sort_values("importance", ascending=False)
    print("\nأهم الميزات:")
    print(feat_imp_df.head(20).to_string(index=False))
except Exception as e:
    print("خطأ عند حساب أهمية الميزات:", e)

# === 11) حفظ الموديل والـ scaler ===
joblib.dump(model, "/content/rf_portscan_model.joblib")
joblib.dump(scaler, "/content/scaler_portscan.joblib")
print("\nتم حفظ الموديل والـ scaler في /content/ (rf_portscan_model.joblib, scaler_portscan.joblib)")


عدد الصفوف: 286467
Label
PortScan    158930
BENIGN      127537
Name: count, dtype: int64

عمود الlabel المستخدم: Label

قيمة_counts للـ labels (أول 50):
Label
PortScan    158930
BENIGN      127537
Name: count, dtype: int64

التسميات التي تم اعتبارها PortScan (مكتشفة تلقائياً): ['PortScan']

توزيع الفئات (PortScan vs Others): {np.int64(0): np.int64(127537), np.int64(1): np.int64(158930)}

الميزات المتاحة من القالب: ['Destination Port', 'Flow Duration', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Total Fwd Packets', 'Flow Bytes/s', 'Flow Packets/s', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets']
لم يتم العثور على Source IP أو Destination Port بأسماء متوقعة -> سيتم تجاهل unique_dst_ports_per_src

نسبة القيم الفارغة لكل feature قبل التعويض:
Destination Port               0.000000
Flow Duration                  0.000000
Init_Win_bytes_forward         0.000000
Init_Win_bytes_backwar

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
